In [142]:
import glob
import monai
import os
from os.path import basename, dirname, exists, isdir, join, split
from pathlib import Path
import functools
import multiprocessing as mp
import os
from functools import partial
import numpy as np
import pandas as pd
from toolz.itertoolz import groupby
import SimpleITK as sitk
import os

root_dir='/workspaces/prost_anatomy_seg/data/550'

def analyze_single_label(uniq_num,centers, big_mask, connected,in_min):
    infered_inner=(connected==uniq_num)
    total=np.sum(infered_inner.flatten())
    inn= np.sum(np.logical_and(infered_inner,big_mask).flatten())/total
    # cov= np.sum(np.logical_and(infered,centers).flatten())/np.sum(centers.flatten())
    res= (inn>in_min) #and (cov>cover_min)
    return res

def get_my_specifity(bi,inn,twos,curr,epoch,folder_path,batch_id,bigger_mask):
    centers= twos[bi,:,:,:]
    inferred=curr[bi,:,:,:]
    big_mask=bigger_mask[bi,:,:,:]

    if(np.sum(inferred.flatten())==0):
        return 1.0

    connected=sitk.GetArrayFromImage(sitk.ConnectedComponent(sitk.GetImageFromArray(inferred.astype(int))))
    uniqq=np.unique(connected)
    uniqq= list(filter(lambda el:el>0,uniqq))
    in_min=0.5
    res= list(map(lambda uniq_num: analyze_single_label(uniq_num,centers, big_mask, connected,in_min), uniqq))
    res= np.mean(np.array(res).astype(int))
    return res

def is_sth_in_areas(uniq_num,arr,inferred):

    bool_arr=(arr.copy()==uniq_num)
    summ=np.sum(inferred[bool_arr].flatten())
    res= summ>0
    return res

def get_connected_components_num(arr):
    connected=sitk.GetArrayFromImage(sitk.ConnectedComponent(sitk.GetImageFromArray(arr.astype(int))))
    uniqq=np.unique(connected)
    uniqq= list(filter(lambda el:el>0,uniqq))
    return len(uniqq)    

def get_my_sensitivity(bi,inn,twos,curr,epoch,folder_path,batch_id,bigger_mask):
    curr_in= inn[bi,:,:,:]
    curr_twos= twos[bi,:,:,:]
    inferred=curr[bi,:,:,:]
    curr_bigger_mask=bigger_mask[bi,:,:,:]
    if(epoch%10==0):
        curr_num=bi*100+batch_id
        # sitk.WriteImage(sitk.GetImageFromArray(curr_curr.astype(np.uint8)), f"{folder_path}/{curr_num}_inferred.nii.gz")
        # sitk.WriteImage(sitk.GetImageFromArray(curr_bigger_mask.astype(np.uint8)), f"{folder_path}/{curr_num}_big_mask.nii.gz")
        # sitk.WriteImage(sitk.GetImageFromArray(curr_twos.astype(np.uint8)), f"{folder_path}/{curr_num}_centers.nii.gz")
        
    
    total = np.sum(curr_in.flatten())
    total_twos = np.sum(curr_twos.flatten())
    curr_percent_in=np.zeros(1)
    curr_percent_covered=np.zeros(1)

    if(total_twos==0):
        return -1.0
    if(total==0):
        return 0.0
    
    components_ones = get_connected_components_num(curr_bigger_mask)
    components_twos = get_connected_components_num(curr_twos)

    arr=curr_bigger_mask
    if(components_twos>components_ones):
        print(f"ones are mergeddd")
        arr=curr_twos

    connected=sitk.GetArrayFromImage(sitk.ConnectedComponent(sitk.GetImageFromArray(arr.astype(int))))
    uniqq=np.unique(connected)
    uniqq= list(filter(lambda el:el>0,uniqq))


    res=np.array(list(map(lambda uniq_num: is_sth_in_areas(uniq_num,connected,inferred),uniqq)))
    res=res.astype(int)
    return np.mean(res.flatten())
    # curr_percent_in=np.sum(curr_in.flatten())/(total)
    # curr_percent_covered=  np.sum(((curr_in) & (curr_twos)).flatten())/ total_twos

    # return (curr_percent_in>0.8 and curr_percent_covered>0.4).astype(float)
   





In [143]:

def group_files(root_dir):
    """ 
    based on name will group the files into triplets of related files
    """
    all_files=os.listdir(root_dir)
    grouped_by_master= groupby(lambda path : path.split('_')[0],all_files)
    return list(dict(grouped_by_master).items())

grouped=group_files(root_dir)    
def chunks(xs, n):
    n = max(1, n)
    return (xs[i:i+n] for i in range(0, len(xs), n))
# aa=list(map(lambda g:analyze_case(g,root_dir)  ,grouped))
grouped=list(chunks(grouped,10))
grouped=sorted(grouped, key=lambda el: el[0])

In [144]:
# percent_in= np.zeros(1)
# percent_out=np.zeros(1)
# percent_covered=np.zeros(1)
# is_correct=np.zeros(1)
# my_sensitivity=np.zeros(1)
# my_specificity=np.zeros(1)

grouped

def getarrs_files(group):
    num,paths=group
    paths= list(map(lambda p: f"{root_dir}/{p}",paths))
    centers=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'centers' in pathh  ,paths))[0])).astype(bool)
    big_mask=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'big_mask' in pathh  ,paths))[0])).astype(bool)
    inferred=sitk.ReadImage(list(filter(lambda pathh: 'inferred' in pathh  ,paths))[0])
    inferred=sitk.GetArrayFromImage(inferred)
    return (centers,big_mask,inferred)

def analyze_cases(group,root_dir):
    percent_in= np.zeros(1)
    percent_out=np.zeros(1)
    percent_covered=np.zeros(1)
    is_correct=np.zeros(1)
    my_sensitivity=np.zeros(1)
    my_specificity=np.zeros(1)

    epoch=5
    batch_id=1


    arrs=list(map(getarrs_files, group))

    centers= np.stack(list(map(lambda el: el[0],arrs)))
    big_mask= np.stack(list(map(lambda el: el[1],arrs)))
    inferred= np.stack(list(map(lambda el: el[2],arrs)))

    shapp=inferred.shape

    bigger_mask= big_mask
    curr=inferred
    
    # curr= torch.sum(curr,dim=1)
    inn = curr & bigger_mask
    twos= centers


    # curr= np.round(np.random.random(curr.shape)).astype(bool) #TODO remove            

        
    
    base='/workspaces/konwersjaJsonData/explore/validation_to_look_into'
    folder_path=f"{base}/{epoch}"
    os.makedirs(folder_path,exist_ok=True)
    
    with mp.Pool(processes = mp.cpu_count()) as pool:
       my_sensitivity=pool.map(partial(get_my_sensitivity,inn=inn,twos=twos,curr=curr,epoch=epoch,folder_path=folder_path,batch_id=batch_id,bigger_mask=bigger_mask),range(shapp[0]))
    
    with mp.Pool(processes = mp.cpu_count()) as pool:
        my_specificity=pool.map(partial(get_my_specifity,inn=inn,twos=twos,curr=curr,epoch=epoch,folder_path=folder_path,batch_id=batch_id,bigger_mask=bigger_mask),range(shapp[0]))
    

    # print(f"rrrrrrr {res}")
    my_sensitivity=list(filter(lambda el: np.array(el).flatten()[0]>-1,my_sensitivity  ))      

    my_sensitivity= np.array(list(map(lambda el: np.mean(np.array(el).flatten()) ,my_sensitivity)))
    my_specificity= np.array(list(map(lambda el: np.mean(np.array(el).flatten()) ,my_specificity)))

    # if(len(my_sensitivity)>0):
    #     my_sensitivity= np.mean(np.array(list(map(lambda el : np.array(np.mean(el)).flatten(),my_sensitivity))))
    #     my_specificity= np.mean(np.array(list(map(lambda el : np.array(np.mean(el)).flatten(),my_specificity))))
    #     is_correct= (my_sensitivity+my_specificity)/2
    
    # total = np.sum(curr.flatten())
    
    # if(total>0):
    #     percent_in=np.sum(inn.flatten())/(total)
    # if(total>0):
    #     out = (curr) & (~bigger_mask)
    #     percent_out= np.sum(out.flatten())/total

    
    # two_sum=np.sum(twos.flatten())
    # if(two_sum>0):
    #     percent_covered=  np.sum(((curr) & (twos)).flatten())/two_sum
    
    # # print(f"tttttt is_correct {is_correct} total {total} ((curr) & (twos)).sum() {((curr) & (twos)).sum()}  (curr) & (~bigger_mask) {((curr) & (~bigger_mask)).sum()}")
    # is_correct=np.array(is_correct).flatten()

    my_sensitivity=np.array([np.mean(my_sensitivity)])
    my_specificity=np.array([np.mean(my_specificity)])

    # percent_in=np.array([percent_in]).flatten()
    # percent_out=np.array([percent_out]).flatten()
    # percent_covered=np.array([percent_covered]).flatten()

    print(f"my_sensitivity {my_sensitivity} my_specificity {my_specificity}")

    return (my_sensitivity,my_specificity)


In [145]:
rrrr= list(map(lambda group: analyze_cases(group,root_dir),grouped))
# #num 808
# # my_sensitivity [0.33333333] my_specificity [0.5]
# group=list(filter(lambda el: int(el[0][0])==808,grouped))[0]
# analyze_cases(group,root_dir)

# # grouped[100][0]
# group

ones are mergeddd


my_sensitivity [0.33333333] my_specificity [0.7]
my_sensitivity [0.75] my_specificity [1.]
my_sensitivity [0.6] my_specificity [1.]
my_sensitivity [0.75] my_specificity [0.85]
ones are mergeddd
my_sensitivity [0.58333333] my_specificity [0.9]
my_sensitivity [0.375] my_specificity [0.8]
my_sensitivity [0.6] my_specificity [0.9]
my_sensitivity [0.6] my_specificity [1.]
my_sensitivity [0.75] my_specificity [0.8]
ones are mergeddd
my_sensitivity [0.73809524] my_specificity [0.9]
my_sensitivity [0.8] my_specificity [1.]
my_sensitivity [0.72222222] my_specificity [0.75]
my_sensitivity [0.91666667] my_specificity [0.95]
my_sensitivity [0.6] my_specificity [0.9]
ones are mergeddd
ones are mergeddd
my_sensitivity [0.71428571] my_specificity [0.9]
my_sensitivity [0.83333333] my_specificity [0.85]
my_sensitivity [0.6] my_specificity [0.7]
my_sensitivity [0.875] my_specificity [0.95]
my_sensitivity [0.66666667] my_specificity [0.8]
my_sensitivity [0.83333333] my_specificity [0.9]
my_sensitivity [0

In [150]:
a = list(map(lambda el: el[0],rrrr))
b = list(map(lambda el: el[1],rrrr))
print(f" sensitivity {np.mean(np.array(a).flatten())} specificity {np.mean(np.array(b).flatten())}")

 sensitivity 0.6783256752993594 specificity 0.8963157894736843


In [146]:
import seaborn as sns
arr=sitk.ReadImage(f"{root_dir}/808_big_mask.nii.gz")
inferred=sitk.GetArrayFromImage(sitk.ReadImage(f"{root_dir}/808_inferred.nii.gz"))
connected=sitk.GetArrayFromImage(sitk.ConnectedComponent(arr))
booll= (connected==3 )

np.sum(inferred[booll].flatten())
connected.shape
# sitk.WriteImage(sitk.ConnectedComponent(arr), '/workspaces/prost_anatomy_seg/data/test.nii.gz')

(28, 48, 56)

In [147]:
tttt=list(map(lambda el: el[0] ,rrrr))
tttt= list(filter(lambda el: el>-1,tttt))
np.nanmean(tttt)

0.6783256752993594

In [148]:
np.mean(aa)


NameError: name 'aa' is not defined

In [ ]:

def get_is_correct(curr_in,curr_twos):
    total = curr_in.sum()
    curr_percent_in=np.zeros(1)
    curr_percent_covered=np.zeros(1)

    if(total.item()>0):
        curr_percent_in=curr_in.sum()/(total)
    if(curr_twos.sum().item()>0):
        curr_percent_covered=  ((curr_in) & (curr_twos)).sum()/ curr_twos.sum()

    return (curr_percent_in>0.8 and curr_percent_covered>0.4)


def analyze_case_sum(group,root_dir):
    num,paths=group
    paths= list(map(lambda p: f"{root_dir}/{p}",paths))

    centers=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'centers' in pathh  ,paths))[0])).astype(bool)
    big_mask=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'big_mask' in pathh  ,paths))[0])).astype(bool)
    inferred=sitk.ReadImage(list(filter(lambda pathh: 'inferred' in pathh  ,paths))[0])
    connected=sitk.GetArrayFromImage(inferred)
    connected=(connected>0)
    # uniqq= list(filter(lambda el:el>0,uniqq))
    cover_min=0.5
    in_min=0.8
    
    inn = connected & big_mask

    res=get_is_correct(inn,centers)
    # res= analyze_single_label(True,centers, big_mask, connected,cover_min,in_min)
    res= np.mean(np.array(res).astype(int))
    return (num,res)

grouped=group_files(root_dir)

bb=list(map(lambda g:analyze_case_sum(g,root_dir)  ,grouped))

In [ ]:
# np.mean(bb)
bbb= list(map(lambda tupl: tupl[1],bb))
# np.mean(bbb)
np.mean(bbb)